Pontificia Universidad Católica de Chile <br>
Departamento de Ciencia de la Computación <br>
IIC2433 - Minería de Datos
<br>

<center>
    <h2> Tarea 2 </h2>
    <h1> Ham o spam  </h1>
    <p>
        Profesor Marcelo Mendoza<br>
        Segundo Semestre 2022<br>    
        Fecha de entrega: Viernes 23 de septiembre 22.00 horas
    </p>
    <br>
</center>

<br>

---

## Indicaciones

Deberás entregar **SOLO** el archivo .ipynb en el buzón respectivo en canvas. 

**IMPORTANTE**: 
- Se te dará puntaje tanto por código como por la manera en la que respondas las preguntas planteadas. Es decir, si tienes un código perfecto pero este no es explicado o no se responden preguntas asociadas a este, no se tendrá el puntaje completo.
- El notebook debe tener todas las celdas de código ejecutadas. Cualquier notebook que no las tenga no podrá ser corregido.
- El carácter de esta tarea es **INDIVIDUAL**. Cualquier instancia de copia resultará en un 1,1 como nota de curso.


Utilizaremos una base de datos ubicada en Kaggle https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset que puedes encontrar igualmente en canvas como csv para descargar.

## Introducción

Nadie es inmune a recibir mensajes de Movistar o Entel ofreciéndonos planes. Abrir un mensaje para encontrarse con una hermosa sorpresa: es spam. 

Este es un problema a nivel mundial, tanto así que se han armado bases de datos con diferentes mensajes de texto recibidos por persona y si son considerados como spam o no (si no son spam se refiere a los mensajes como ham).

Utilizando la vectorización de frases y clusterizando estas, deberás predecir si esta es o no spam. Además, deberás obtener los índices de calidad de los clusters.

## 0. Setup

In [1]:
from IPython.display import clear_output

!pip3 uninstall spacy
!pip3 install spacy

!spacy download en_core_web_lg
clear_output()
print('Ahora debes reiniciar el entorno de ejecución y ejecutar a partir de la siguiente celda')

Ahora debes reiniciar el entorno de ejecución y ejecutar a partir de la siguiente celda


## 1. Importar librerías y descargar dataset
En esta tarea trabajaremos con la librería `spacy` y el pipeline `en_core_web_lg` el cual pesa más de 500 MB y contiene un vocabulario en inglés de más de medio millón de palabras. Cada una de estas palabras es representable a partir de un vector de 300 dimensiones que nos ayudarán en la tarea. Revisa la [documentación](https://spacy.io/api) documentación de la librería para saber más.

In [193]:
import pandas as pd
import numpy as np
from collections import defaultdict
import spacy

nlp = spacy.load("en_core_web_lg")

### Leer dataset

In [194]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [195]:
url = '/gdrive/My Drive/Colab Notebooks/Minería de Datos/Tareas/T2/spam.csv'
df = pd.read_csv(url, index_col=0, encoding="ISO-8859-1")
df

,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
v1,,,,
ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...
spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN


## 2. Procesamiento de los datos (1 punto)

### 2.1 Eliminación de datos
Solamente analizaremos las columnas de si es o no spam y cuál es el mensaje. Elimina las columnas restantes y preprocesa las filas eliminando los valores nulos.

In [196]:
df = df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)
df = df.dropna()
df.head(5)

,v2
v1,
ham,"Go until jurong point, crazy.. Available only ..."
ham,Ok lar... Joking wif u oni...
spam,Free entry in 2 a wkly comp to win FA Cup fina...
ham,U dun say so early hor... U c already then say...
ham,"Nah I don't think he goes to usf, he lives aro..."


### 2.2 Preprocesamiento de oraciones

Acá te damos el código para preprocesar un texto.

In [197]:
import string

all_stopwords = nlp.Defaults.stop_words

def remove_punctuation(text):
  text = [token for token in text if not token.is_punct]
  return text

def remove_stopwords(words):
  words = [word for word in words if not word in all_stopwords]
  return words

def lemmatize(words):
  words = [word.lemma_ for word in words]
  return words

def remove_non_alpha(words):
  words = [word for word in words if word.isalpha()]
  return words

def lower(words):
  words = [word.lower() for word in words]
  return words

def min_len(words, length=3):
  words = [word for word in words if len(word)>=length]
  return words

def preprocess(text):

  doc = nlp(text)
  tokens = remove_punctuation(doc)
  tokens = remove_stopwords(tokens)
  tokens = lemmatize(tokens)
  tokens = remove_non_alpha(tokens)
  tokens = lower(tokens)
  tokens = min_len(tokens, length=3)

  return ' '.join(tokens).strip()

# Este es un ejemplo para que veas si tu preprocesamiento funcionó.
new_text = preprocess("This is the 2nd time we have tried 2 contact...")
new_text

'this the time have try contact'

Preprocesa todos los mensajes utilizando el método preprocess y guárdalos en un dataframe.

In [198]:
df_p = df
df_p['v2'] = df['v2'].apply(func=preprocess)

df_p

,v2
v1,
ham,until jurong point crazy available only bugis ...
ham,lar joke wif oni
spam,free entry wkly comp win cup final tkts may te...
ham,dun say early hor already then say
ham,nah not think usf live around here though
...,...
spam,this the time have try contact have win the po...
ham,will esplanade home
ham,pity mood for that any other suggestion


### 2.3 Vectorizar oraciones
En esta tarea, el vector de una oración será el promedio de los vectores de cada una de las palabras que fueron preprocesadas de la oración. La función presentada a continuación vectoriza una oración a partir de los vectores de las palabras.



In [199]:
def sentence_vector(text):
  text = nlp(text)
  vectores = []
  for t in text:
    t_vector = t.vector
    vectores.append(t_vector)
  return np.array(vectores).sum(axis=0)/len(vectores)

In [200]:
df_p['v2'] = df['v2'].apply(func=sentence_vector)

### 2.4. Obtener matriz de distancias


Obtén una forma de calcular una matriz que por cada par distintos de oraciones contenga la distancia euclidiana y coseno entre los vectores que representan a cada una. 

Hint: el método pairwise_distances de sklearn realiza esta operación eficientemente y no genera problemas de RAM.



Creamos dos función a la cual se le pasen dos vectores y nos proporcione una matriz de distancias. Una se basa en el método euclideano y la otra por coseno

In [201]:
from sklearn.metrics import pairwise_distances
def matriz_distancias_e(v1, v2):
  matriz = pairwise_distances(v1, v2, metric='euclidean')
  return np.array(matriz)
def matriz_distancias_c(v1, v2):
  matriz = pairwise_distances(v1, v2, metric='cosine')
  return np.array(matriz)

Eliminamos datos nulos

In [202]:
df_p = df_p.dropna()

Creamos na variable que contenga los vectores

In [203]:
vectors = np.stack(df_p["v2"].values)

Generamos una matriz de distancias euclideanas

In [204]:
m_e = matriz_distancias_e(vectors, vectors)
m_e

array([[ 0.      , 22.287176, 21.584124, ..., 25.64222 , 17.197527,
        41.088566],
       [22.287176,  0.      , 29.280226, ..., 36.931423, 31.36411 ,
        45.26217 ],
       [21.584124, 29.280226,  0.      , ..., 29.595678, 25.300081,
        43.817024],
       ...,
       [25.64222 , 36.931423, 29.595678, ...,  0.      , 20.358713,
        42.429527],
       [17.197527, 31.36411 , 25.300081, ..., 20.358713,  0.      ,
        43.578625],
       [41.088566, 45.26217 , 43.817024, ..., 42.429527, 43.578625,
         0.      ]], dtype=float32)

Generamos una matriz con las distancias por coseno

In [205]:
m_c = matriz_distancias_c(vectors, vectors)
m_c

array([[0.        , 0.8346631 , 0.42814863, ..., 0.3057986 , 0.15957665,
        0.64997876],
       [0.8346631 , 0.        , 1.0922576 , ..., 0.9585984 , 0.8705199 ,
        0.99595284],
       [0.42814863, 1.0922576 , 0.        , ..., 0.4369179 , 0.40039504,
        0.7226788 ],
       ...,
       [0.3057986 , 0.9585984 , 0.4369179 , ..., 0.        , 0.18286031,
        0.5653542 ],
       [0.15957665, 0.8705199 , 0.40039504, ..., 0.18286031, 0.        ,
        0.6536323 ],
       [0.64997876, 0.99595284, 0.7226788 , ..., 0.5653542 , 0.6536323 ,
        0.        ]], dtype=float32)

In [206]:
m_e.shape[0]

5537

## 3. Clase AgglomerativeClustering (3 puntos)

Esta clase debe implemetar el algoritmo de clustering jerárquico aglomerativo. Para esto **debes** programar los siguientes métodos:


### **\_\_init\_\_**
Inicializa el algoritmo a partir de: la matriz `X` de los mensajes y la matriz de distancias. Considera además todas las variables que necesites a lo largo de la ejecución de tu algoritmo, se recomienda como mínimo:
*   Un contador que indique el nivel actual de aglomeración.
*   Un diccionario o lista que almacene los clusters en cada nivel de aglomeración. Inicialmente, en el nivel 0, existe un *cluster* por cada mensaje de `X`.
*   La matriz de distancia entre los *clusters* donde el elemento `matriz[id1][id2]` corresponde a la distancia entre los clusters con identificadores `id1` e `id2` respectivamente. 
*   Una copia de la matriz original X.


### **clusterize**
Ejecuta el método next_level hasta que solo existan dos *clusters*.

##### **next_level**
Equivale a realizar un nivel de aglomeración del algoritmo. A modo general deben:
1.   Obtener el par de *clusters* con menor distancia a partir de la matriz de distancias obtenida en 3.
2.   Unir ambos *clusters*.
3.   Guardar el nuevo conjunto de *clusters* correspondientes al nivel actual de aglomeración.
4.   Actualizar la matriz de distancias según el nuevo conjunto de *clusters*.

##### **update_matrix**
Actualiza la matriz de distancias dado un nuevo *cluster*. La distancia entre *clusters* debe poder calcularse según los siguientes enlaces (`linkage`) vistos en clases:
1.   **centroid**: distancia entre medias.
2.   **single**: simple.

<br>

---

**NOTA**: puedes entregarle los argumentos que quieras a estos métodos y tambien crear otros métodos que consideres pertinentes.

En base a lo visto en clases deberás implementar el algoritmo de clustering aglomerativo para agrupar los datos previamente preprocesados.

Se utiliza esto como inspiración: https://github.com/mithunjmistry/hierarchical-clustering/blob/master/HierarchicalClustering.ipynb

In [207]:
class AgglomerativeClustering:

  def __init__(self, X, linkage="centroid", distance="Euclidean"):
    # Vectores de cada pais. 
    self.X = X.copy()
    # Linkage.
    self.linkage = linkage
    # Métrica de distancia.
    self.distance = distance

    # Matriz de distancias (obtén la matriz de distancias inicial)
    self.matrix = self.get_distance(X, X)

    # Copia de la matriz de distancias original.
    self.original_matrix = self.matrix.copy()

    # Contador nivel actual aglomeración
    self.nivel_aglomeracion = 0

    # Diccionario que guarda los clusters en el respectivo nivel
    self.dict_na = {}

  def clusterize(self):
    # Inicializamos un dataframe que contendrá las posiciones de los clusters en la matriz.
    df = pd.DataFrame(data=np.ones(self.X.shape[0])*np.inf)
    for i in range(0,self.matrix.shape[0]):
      # i será nuestro contador de aglomeraciones
      self.nivel_aglomeracion = i
      # iteramos para obtener los clusters
      cluster_matrix = self.next_level(df)

    return self.dict_na, cluster_matrix

  def next_level(self, df):
    
    # Se obtiene el par de clusters con menor distancia de la matriz de distancias.
            ij_min = np.unravel_index(self.matrix.argmin(), self.matrix.shape) 
            #np.unravel_index nos da la posición de el valor mínimo
            # Ahora completaremos el dataframe combinando los dos puntos y clusters
            if self.nivel_aglomeracion == 0:
                df.iloc[ij_min[0]] = 0
                df.iloc[ij_min[1]] = 0
            else:
                try:
                    a = int(df.iloc[ij_min[0]])
                except:
                    df.iloc[ij_min[0]] = self.nivel_aglomeracion
                    a = self.nivel_aglomeracion
                try:
                    b = int(df.iloc[ij_min[1]])
                except:
                    df.iloc[ij_min[1]] = self.nivel_aglomeracion
                    b = self.nivel_aglomeracion
                df[(df[0]==a) | (df[0]==b)] = self.nivel_aglomeracion
            # Almacenamos el nivel y los clusters que existen
            self.dict_na[self.nivel_aglomeracion] = df[0].unique()     

            #De esta manera si combinamos dos puntos, estos quedarán anotados en el diccionario con su respectivo cluster.
            for j in range(0, ij_min[0]):
                #Ignoramos la diagonal transformando 0 a inf
                if np.isfinite(self.matrix[ij_min[0]][j]) and np.isfinite(self.matrix[ij_min[1]][j]):
                    #Actualizamos la matriz
                    self.update_matrix(ij_min, j)
            # Para evitar futuras combinaciones hacemos todo infinito
            self.matrix[ij_min[0]] = np.inf

            return df[0]
        

  
  def update_matrix(self, cluster, j):

    if self.linkage == "centroid":
      # Tomamos el promedio
      self.matrix[cluster[1]][j] = (self.matrix[cluster[0]][j] + self.matrix[cluster[1]][j])/2.0          
      return self.matrix[cluster[1]][j]

    elif self.linkage == "single":
      # Tomamos el mínimo
      self.matrix[cluster[1]][j] = min(self.matrix[cluster[0]][j], self.matrix[cluster[1]][j])
      return self.matrix[cluster[1]][j]

  def get_distance(self, vector1, vector2):
    if self.distance == "Euclidean": 
      # Obtenemos la distancia
      matrix = matriz_distancias_e(vector1, vector2)
      # Como es simetrica, solo dejamos el triangulo de abajo
      matrix = np.tril(matrix)
      # Reemplazamos os 0 por inf, lo que hará más fácil sacar mínimos
      matrix[matrix == 0] = np.inf
      return matrix
    
    elif self.distance == "Cosine":
      matrix = matriz_distancias_c(vector1, vector2)
      # Como es simetrica, solo dejamos el triangulo de abajo
      matrix = np.tril(matrix)
      # Reemplazamos os 0 por inf, lo que hará más fácil sacar mínimos
      matrix[matrix == 0] = np.inf
      return matrix

Utiliza la clase para realizar la aglomeración con los datos preprocesados de spam:

In [208]:
clustering = AgglomerativeClustering(vectors)
d, target = clustering.clusterize()

d es un diccionario con los momentos de aglomeracion y los clusters. Target son los clusters al final de todo el proceso y en el último momento de clusterización.

In [209]:
for i in range(50):
  print(d[i])

[inf  0.]
[inf  0.  1.]
[inf  0.  2.]
[inf  0.  3.]
[inf  0.  4.]
[inf  5.  0.  4.]
[inf  6.  0.  4.]
[inf  6.  0.  7.  4.]
[inf  6.  8.  0.  7.  4.]
[inf  6.  8.  0.  7.  9.  4.]
[inf  6.  8.  0.  7.  9. 10.  4.]
[inf  6.  8.  0.  7.  9. 10. 11.  4.]
[inf  6.  8. 12.  0.  7.  9. 10. 11.  4.]
[inf  6.  8. 12.  0.  7.  9. 13. 10. 11.  4.]
[inf 14.  6.  8. 12.  0.  7.  9. 13. 10. 11.  4.]
[inf 14.  6.  8. 12.  0.  7.  9. 13. 10. 15.  4.]
[inf 14.  6.  8. 12.  0.  7.  9. 13. 10. 15.  4. 16.]
[inf 14.  6.  8. 17. 12.  0.  7.  9. 13. 10. 15.  4. 16.]
[inf 14.  6.  8. 17. 12.  0.  7.  9. 13. 10. 15.  4. 16. 18.]
[inf 14.  6.  8. 19. 17. 12.  0.  7.  9. 13. 10. 15.  4. 16. 18.]
[inf 14.  6.  8. 20. 17. 12.  0.  7.  9. 13. 10. 15.  4. 16. 18.]
[inf 21. 14.  6.  8. 20. 17. 12.  0.  7.  9. 13. 10. 15.  4. 16. 18.]
[inf 21. 14.  6.  8. 20. 22. 17. 12.  0.  7.  9. 13. 10. 15.  4. 16. 18.]
[inf 23. 21. 14.  6.  8. 20. 22. 17. 12.  0.  7.  9. 13. 10. 15.  4. 16.
 18.]
[inf 23. 21. 24. 14.  6.  8. 20

Podemos ver como se comienza con un solo cluster y luego mientras más iteraciones, más clusters

In [210]:
target

0       5536.0
1       5536.0
2       5536.0
3       5536.0
4       5536.0
         ...  
5532    5536.0
5533    5536.0
5534    5536.0
5535    5536.0
5536    5536.0
Name: 0, Length: 5537, dtype: float64

# 4. Comparación con distintos parámetros (1 punto)

En esta parte deberás comparar distintas configuraciones de tu algoritmo de *clustering* y concluir cual de estas es la mejor.

Una forma de comparar *clusters* es a partir de su *silhouette score*. Este mide cuán similar es un objeto a su propio *cluster* (cohesión) en comparación con otros *clusters* (separación). Completa el siguiente código utilizando la función `silhouette_score` de `sklearn.metrics`.

NOTA: debes adaptar la estructura de clusters retornada por `AgglomerativeClustering` de tal forma que pueda ser utilizada como los `labels` que recibe `silhouette_score` ([documentación](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html)).

Adaptaremos la clase para poder obtener los datos de labels para el  silhouette_score. Definiremos la cantidad de aglomeraciones previamente. Calcularemos 1000, ya que más que estas nos harán sumamente costoso computacionalmente el ejercicio y no darán un resultado relevante.

In [211]:
class AgglomerativeClustering:

  def __init__(self, X, linkage="centroid", distance="Euclidean", cutoff=1000):
    # Vectores de cada pais. 
    self.X = X.copy()
    # Linkage.
    self.linkage = linkage
    # Métrica de distancia.
    self.distance = distance

    # Matriz de distancias (obtén la matriz de distancias inicial)
    self.matrix = self.get_distance(X, X)

    # Copia de la matriz de distancias original.
    self.original_matrix = self.matrix.copy()

    # Contador nivel actual aglomeración
    self.nivel_aglomeracion = 0

    # Diccionario que guarda los clusters en el respectivo nivel
    self.dict_na = {}

    # Numero máximo de iteraciones
    self.cutoff = cutoff

  def clusterize(self):
    # Inicializamos un dataframe que contendrá las posiciones de los clusters en la matriz.
    df = pd.DataFrame(data=np.ones(self.X.shape[0])*np.inf)
    for i in range(0,self.cutoff):
      # i será nuestro contador de aglomeraciones
      self.nivel_aglomeracion = i
      # iteramos para obtener los clusters
      cluster_matrix = self.next_level(df)

    return self.dict_na, cluster_matrix

  def next_level(self, df):
    
    # Se obtiene el par de clusters con menor distancia de la matriz de distancias.
            ij_min = np.unravel_index(self.matrix.argmin(), self.matrix.shape) 
            #np.unravel_index nos da la posición de el valor mínimo
            # Ahora completaremos el dataframe combinando los dos puntos y clusters
            if self.nivel_aglomeracion == 0:
                df.iloc[ij_min[0]] = 0
                df.iloc[ij_min[1]] = 0
            else:
                try:
                    a = int(df.iloc[ij_min[0]])
                except:
                    df.iloc[ij_min[0]] = self.nivel_aglomeracion
                    a = self.nivel_aglomeracion
                try:
                    b = int(df.iloc[ij_min[1]])
                except:
                    df.iloc[ij_min[1]] = self.nivel_aglomeracion
                    b = self.nivel_aglomeracion
                df[(df[0]==a) | (df[0]==b)] = self.nivel_aglomeracion
            # Almacenamos el nivel y los clusters que existen
            self.dict_na[self.nivel_aglomeracion] = df[0].unique()     

            #De esta manera si combinamos dos puntos, estos quedarán anotados en el diccionario con su respectivo cluster.
            for j in range(0, ij_min[0]):
                #Ignoramos la diagonal transformando 0 a inf
                if np.isfinite(self.matrix[ij_min[0]][j]) and np.isfinite(self.matrix[ij_min[1]][j]):
                    #Actualizamos la matriz
                    self.update_matrix(ij_min, j)
            # Para evitar futuras combinaciones hacemos todo infinito
            self.matrix[ij_min[0]] = np.inf

            return df[0]
        

  
  def update_matrix(self, cluster, j):

    if self.linkage == "centroid":
      # Tomamos el promedio
      self.matrix[cluster[1]][j] = (self.matrix[cluster[0]][j] + self.matrix[cluster[1]][j])/2.0          
      return self.matrix[cluster[1]][j]

    elif self.linkage == "single":
      # Tomamos el mínimo
      self.matrix[cluster[1]][j] = min(self.matrix[cluster[0]][j], self.matrix[cluster[1]][j])
      return self.matrix[cluster[1]][j]

  def get_distance(self, vector1, vector2):
    if self.distance == "Euclidean": 
      # Obtenemos la distancia
      matrix = matriz_distancias_e(vector1, vector2)
      # Como es simetrica, solo dejamos el triangulo de abajo
      matrix = np.tril(matrix)
      # Reemplazamos os 0 por inf, lo que hará más fácil sacar mínimos
      matrix[matrix == 0] = np.inf
      return matrix
    
    elif self.distance == "Cosine":
      matrix = matriz_distancias_c(vector1, vector2)
      # Como es simetrica, solo dejamos el triangulo de abajo
      matrix = np.tril(matrix)
      # Reemplazamos os 0 por inf, lo que hará más fácil sacar mínimos
      matrix[matrix == 0] = np.inf
      return matrix

Obtenemos las iteraciones con los clusters respectivos

In [212]:
clustering = AgglomerativeClustering(vectors)
d, target = clustering.clusterize()

Obtenemos una tabla con 1 para cuando el mensaje es ham y 0 cuando es spam

In [213]:
X = df_p.index
X = pd.get_dummies(X)

Formateamos target para que entre bien en el algoritmo

In [214]:
labels = target.values
labels[labels==np.inf] = -1
labels = labels.astype(int)

In [215]:
from sklearn.metrics import silhouette_score

def messages_silhouette_score(X, clusters):
  print('Silhouette score for 1000 is '+str(silhouette_score(X, clusters, metric='euclidean')))

In [216]:
messages_silhouette_score(X, labels)

Silhouette score for 1000 is -0.8060321473722232


Ahora realiza una busqueda de hiperparámetros para encontrar la configuración que retorne el mejor *silhouette_score*. Como mínimo debes probar todas las combinaciones posibles de los siguientes parámetros:

*   ***Linkage***: centroid y single.
*   ***Distance***: euclidean y cosine.

Obtenemos las variables para las 4 combinaciones y el score

In [217]:
# Centroid, Euclidean
clustering = AgglomerativeClustering(X = vectors, linkage="centroid", distance="Euclidean")
d_ce, target_ce = clustering.clusterize()
labels = target_ce.values
labels[labels==np.inf] = -1
labels = labels.astype(int)
messages_silhouette_score(X, labels)

# Centroid, Cosine
clustering = AgglomerativeClustering(X = vectors, linkage="centroid", distance="Cosine")
d_cc, target_cc = clustering.clusterize()
labels_cc = target_cc.values
labels_cc[labels_cc==np.inf] = -1
labels_cc = labels_cc.astype(int)
messages_silhouette_score(X, labels_cc)

# Single, Euclidean
clustering = AgglomerativeClustering(X = vectors, linkage="single", distance="Euclidean")
d_se, target_se = clustering.clusterize()
labels = target_se.values
labels[labels==np.inf] = -1
labels = labels.astype(int)
messages_silhouette_score(X, labels)

# Single, Cosine
clustering = AgglomerativeClustering(X = vectors, linkage="single", distance="Cosine")
d_sc, target_sc = clustering.clusterize()
labels = target_sc.values
labels[labels==np.inf] = -1
labels = labels.astype(int)
messages_silhouette_score(X, labels)

Silhouette score for 1000 is -0.8060321473722232
Silhouette score for 1000 is -0.7778580458732165
Silhouette score for 1000 is -0.8331226295828066
Silhouette score for 1000 is -0.8081993859490699


Hecho esto, responde las siguientes preguntas. Debes fundamentar todas tus respuestas con los resultados obtenidos en la búsqueda de hiperparametros.
1.   ¿Cual configuración fue la mejor? 
2.   ¿Que métrica de distancia da mejores resultados? (puedes comparar las métricas fijando un valor) 
3.   ¿Que relación observas entre el método de enlace y la métrica de distancia utilizada? **Justifica**.


1.   Podemos ver que el Silhouette Score más cercano a 1 es el de la combinación centroide coseno, por lo que esta es la mejor combinación y la que obtiene los mejores clusters.
2.   La mejor métrica de distancias es la de Coseno en este caso, ya que comparando las combinaciones de coseno con las euclideanas, siempre coseno fue más cercano a 1.
3.   Se puede ver que en el caso de el uso de euclidean es 0.02709 mejor con centroide que con simple. En el caso de cosine es 0.03034 mejor con centroide que con simple. Con esto podemos decir que el método de enlace por centroides es mejor que simple y en un 0.03 aproximadamente.



# 5. Visualización (1 punto)

Los [dendrogramas](https://es.wikipedia.org/wiki/Dendrograma) son una forma muy útil de visualizar el funcionamiento de los algoritmos de *clustering* aglomerativo. Para completar esta sección debes generar un dendrograma a partir de tu mejor configuración de `AgglomerativeClustering`. Para esto debes utilizar la función `dendrogram` del módulo `cluster.hierarchy` de scipy.

<br>
<center>
<img src="https://docs.scipy.org/doc/scipy/_images/scipy-cluster-hierarchy-dendrogram-1_00.png" width="400"/>

Fuente: https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html

</center>

NOTA: Debes investigar que formato tiene la matriz `Z` (*linkage matrix*) que recibe `dendrogram` y adaptar el output de tu algoritmo acordemente. Está estrictamente prohibido obtener `Z` a partir de la función `linkage` del módulo `cluster.hierarchy`. Puedes modificar la clase `AgglomerativeClustering` si lo consideras necesario.

In [218]:
from scipy.cluster.hierarchy import dendrogram

In [224]:
Z = pd.DataFrame(X)
Z[1] = labels_cc

In [220]:
m_c = matriz_distancias_c(Z, Z)

In [225]:
Z = Z.values
Z = np.asmatrix(Z)
Z = Z.transpose

In [ ]:
dendogram(Z)